In [166]:
import pandas as pd
from datetime import date, datetime
import re
import os
#Carregando arquivo excel em formato dataframe
dataframe_original = pd.read_excel('Amostras.xlsx', engine='openpyxl') 
dataframe_teste = pd.read_excel('Dados_Amostra_Teste.xlsx', engine='openpyxl') 

#dataframe_original.head(5)

In [167]:
#Separando apenas as colunas 'DS_EVOLUCAO' e 'VALIDACAO'
dataframe_formatado = dataframe_original[['DS_EVOLUCAO','VALIDACAO']].copy()
#Converto os valores da coluna VALIDACAO SIM para S e NÃO para N
dataframe_formatado.loc[dataframe_formatado['VALIDACAO'] == "SIM", 'VALIDACAO'] = "S"
dataframe_formatado.loc[dataframe_formatado['VALIDACAO'] == "NÃO", 'VALIDACAO'] = "N"
#Removo todos os caracteres '?'
dataframe_formatado['DS_EVOLUCAO'] = dataframe_formatado['DS_EVOLUCAO'].str.replace(r'[^a-zA-Z0-9\s]',' ',regex=False) #

#dataframe_formatado.head(5)

In [168]:
dataframe_teste_formatado = dataframe_teste[['DS_EVOLUCAO','VALIDACAO']].copy()

dataframe_formatado['DS_EVOLUCAO'] = dataframe_formatado['DS_EVOLUCAO'].str.replace(r'[^a-zA-Z0-9\s]',' ',regex=False) #
dataframe_teste_formatado['DS_EVOLUCAO'] = dataframe_teste_formatado['DS_EVOLUCAO'].str.replace(r'[^a-zA-Z0-9\s]',' ',regex=False) #


In [169]:
def remove_special_characters(texto):
    texto_sem_caracteres = re.sub(r'[^a-zA-Z0-9\s]',' ',texto)
    texto_formatado = re.sub(r'[0-9]','',texto_sem_caracteres)
    return texto_formatado

dataframe_formatado['DS_EVOLUCAO'] = dataframe_formatado['DS_EVOLUCAO'].apply(remove_special_characters)
dataframe_teste_formatado['DS_EVOLUCAO'] = dataframe_teste_formatado['DS_EVOLUCAO'].apply(remove_special_characters)

In [170]:
dicionario_positivos = [
    "ASPIROU","ASPIREI","ASPIRO","ASPIRADO","ASPIRAR",
    "ASPIRACAO","SECRECAO","TRACHCARE","VAS","TRACH","CARE",
    "VIAS SUPERIORES","AEREAS","SECRECAO","ASP","SISTEMA FECHADO",
    "VENTILACAO MECANICA","TOT","TQT", "SISTEMA ASPIRACAO","SISTEMA","FECHADO","ENTUBADO","INTUBADO"
]

dicionario_negativos = [
    "EUPNEICO", "AR AMBIENTE","EUPNEICA",
    "EUPENEICA", "EUPENEICO", "EUPONEICO","BRONCOSAPIRACAO"
]

stopwords = ["EM", "DE"]

pattern_stopwords = r"\b(" + "|".join([re.escape(i) for i in stopwords]) + r")\b"
pattern_positivo = r"\b(" + "|".join([re.escape(i) for i in dicionario_positivos]) + r")\b"
pattern_negativo = "|".join([re.escape(i) for i in dicionario_negativos])


In [171]:
#Função para extrair X palavras pré e Y palavras pós a cada termo encontrado na evolução
def extrair_contexto_por_palavras(texto, pre=4, pos=3):
    if not isinstance(texto, str):
        return None
    # separa em palavras
    palavras = texto.split()
    
    resultados_positivos = []
    resultados_negativos = []

    # percorre palavras
    i = 0
    while i < len(palavras):
        # verifica se o termo positivo bate a partir da posição i
        for p in palavras:
            if p in dicionario_negativos:
                resultados_negativos.append(p)
        for p in dicionario_positivos:
            termo_palavras = p.split()
            tamanho = len(termo_palavras)
            if palavras[i:i+tamanho] == termo_palavras:
                # pega 'pre' palavras antes e 'pos' palavras depois
                inicio = max(0, i - pre)
                fim = min(len(palavras), i + tamanho + pos)
                contexto = " ".join(palavras[inicio:fim])
                resultados_positivos.append(contexto)
                i += tamanho - 1  # pula as palavras do termo encontrado
                break
        i += 1
    if resultados_positivos:
     return " ".join(resultados_positivos)

    elif resultados_positivos:
        return " ".join(resultados_negativos)
    else:
        return " ".join(palavras)

In [172]:
dataframe_formatado['DS_CONTEXTO'] = dataframe_formatado['DS_EVOLUCAO'].apply(extrair_contexto_por_palavras)

dataframe_teste_formatado['DS_CONTEXTO'] = dataframe_teste_formatado['DS_EVOLUCAO'].apply(extrair_contexto_por_palavras)



In [173]:
dataframe_formatado['VALIDACAO'].value_counts()
dataframe_teste_formatado['VALIDACAO'].value_counts()

VALIDACAO
N    796
S    230
Name: count, dtype: int64

In [174]:
termo_a_mudar = ["TOT",'TQT']
padrao = "|".join(termo_a_mudar)
dataframe_editar = dataframe_formatado.copy()
dataframe_editar.loc[(dataframe_editar['VALIDACAO'] == 'N') &
                        (dataframe_editar['DS_CONTEXTO'].str.contains(padrao)), 'VALIDACAO'] = "S"

In [175]:
dataframe_teste_mudar = dataframe_teste_formatado.copy()
dataframe_teste_mudar.loc[(dataframe_teste_mudar['VALIDACAO'] == 'N') &
                        (dataframe_teste_mudar['DS_CONTEXTO'].str.contains(padrao)), 'VALIDACAO'] = "S"

In [176]:
dataframe_editar['VALIDACAO'].value_counts()

VALIDACAO
N    982
S    601
Name: count, dtype: int64

In [177]:

path = os.path.join('Dados_Tratados/')
file_name = 'Dados_Tratados_Amostra_' + f"{datetime.today().strftime('%d-%m-%y')}" + ".xlsx"
file_path = path + file_name
dataframe_editar.to_excel(file_path,index=False)

In [178]:

path = os.path.join('Dados_Tratados/')
file_name = 'Dados_Tratados_Teste_' + f"{datetime.today().strftime('%d-%m-%y')}" + ".xlsx"
file_path = path + file_name
dataframe_teste_formatado.to_excel(file_path,index=False)

In [179]:
path = os.path.join('Dados_Tratados/')
file_name = 'Dados_Tratados_Teste_Ajustado_' + f"{datetime.today().strftime('%d-%m-%y')}" + ".xlsx"
file_path = path + file_name
dataframe_teste_mudar.to_excel(file_path,index=False)